# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])
df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
l_d =df_youtube.to_dict(orient='records')

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [6]:
collection.delete_many({})
collection.insert_many(l_d)

## Question 1  

In [ ]:
 cur = collection.find({"channel_title": "Apple"})
list(cur)

## Question 2

In [36]:
cur = collection.aggregate([{"$group":{"_id":"$category_name"}},{"$count":"category_name"}])
list(cur)

[{'category_name': 16}]

## Question 3

In [38]:
for video in l_d:
    listTag = video['tags'].split('|')
    collection.update_one({"_id":{"$eq":video['_id']}},{ "$set":{"tags":listTag}});

## Question 4

In [40]:
cur = collection.find().sort([("views", -1)])
list(cur)

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'],
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category_id': 

## Question 5

In [42]:
cur = collection.aggregate([{"$group":{"_id":"$category_name", "averageViews" : {"$avg":"$views"}}}])
list(cur)

[{'_id': 'People & Blogs', 'averageViews': 892992.28515625},
 {'_id': 'Music', 'averageViews': 1442240.943283582},
 {'_id': 'Education', 'averageViews': 550585.2659574468},
 {'_id': 'Film & Animation', 'averageViews': 966547.2871287129},
 {'_id': 'Shows', 'averageViews': 8726.0},
 {'_id': 'Autos & Vehicles', 'averageViews': 622727.8648648649},
 {'_id': 'Sports', 'averageViews': 618074.711409396},
 {'_id': 'Gaming', 'averageViews': 777213.0357142857},
 {'_id': 'Comedy', 'averageViews': 1317200.7289719626},
 {'_id': 'News & Politics', 'averageViews': 533575.1262626263},
 {'_id': 'Nonprofits & Activism', 'averageViews': 681178.2},
 {'_id': 'Pets & Animals', 'averageViews': 633906.4210526316},
 {'_id': 'Entertainment', 'averageViews': 1059427.7735849055},
 {'_id': 'Science & Technology', 'averageViews': 861563.6158940397},
 {'_id': 'Travel & Events', 'averageViews': 426492.22222222225},
 {'_id': 'Howto & Style', 'averageViews': 556002.6679389313}]

## Question 6 

In [43]:
cur = collection.aggregate([{"$group":{"_id":"$channel_title", "averageLikes" : {"$avg":"$likes"}}},{"$sort":{"averageLikes": -1}}])
list(cur)

[{'_id': 'ibighit', 'averageLikes': 1533457.0},
 {'_id': 'ZaynVEVO', 'averageLikes': 1431683.0},
 {'_id': 'BETNetworks', 'averageLikes': 982842.0},
 {'_id': 'melanie martinez', 'averageLikes': 921577.0},
 {'_id': 'jypentertainment', 'averageLikes': 793387.0},
 {'_id': 'Logan Paul Vlogs', 'averageLikes': 724478.5},
 {'_id': 'Wengie', 'averageLikes': 656526.0},
 {'_id': 'Alesso', 'averageLikes': 618436.0},
 {'_id': 'shakiraVEVO', 'averageLikes': 578266.0},
 {'_id': 'TaylorSwiftVEVO', 'averageLikes': 508015.0},
 {'_id': 'Ed Sheeran', 'averageLikes': 461378.0},
 {'_id': 'PewDiePie', 'averageLikes': 438273.0},
 {'_id': 'jbalvinVEVO', 'averageLikes': 430513.0},
 {'_id': 'Dude Perfect', 'averageLikes': 406305.5},
 {'_id': 'Linkin Park', 'averageLikes': 398487.0},
 {'_id': 'Liza Koshy', 'averageLikes': 397011.2},
 {'_id': 'Conor Maynard', 'averageLikes': 389273.0},
 {'_id': 'Bhad Bhabie', 'averageLikes': 379224.0},
 {'_id': 'Gabbie Hanna', 'averageLikes': 359882.0},
 {'_id': 'Primitive Technol